In [1]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pickle

In [3]:
FILE_PATH = 'annotations_gpt-4/annotations_gpt-4'

with open(f'{FILE_PATH}/experiment_type_to_prolific_id.json') as file:
    experiment_types = json.load(file)

website_preferences = experiment_types['website_preferences']

query_to_time = []
for query_type in website_preferences:
    print("QUERY TYPE:", query_type)
    query_avgs = []

    if query_type == "Non-interactive":
        continue

    for i, prolific_id in enumerate(website_preferences[query_type]):
        if (i == 0): continue
        with open(f'{FILE_PATH}/{prolific_id}.json') as file:
            person_data = json.load(file)

        history = person_data['conversation_history']

        assistant_message = None
        for item in history:
            if item['sender'] == 'user':
                if assistant_message is None:
                    continue
                time_spent = item['time_spent_ms'] / 1000
                query_to_time.append({
                    "question": assistant_message,
                    "response_time": time_spent,
                    "prolific_id": prolific_id
                })
                assistant_message = None
            else: # item['sender'] == 'assistant'
                assistant_message = item['message']

with open('query_to_time_dataset.pkl', 'wb') as file:
    pickle.dump(query_to_time, file)


QUERY TYPE: Supervised Learning
QUERY TYPE: Non-interactive
QUERY TYPE: Pool-based Active Learning
QUERY TYPE: Generative edge cases
QUERY TYPE: Generative yes/no questions
QUERY TYPE: Generative open-ended questions


### Encoding all sentences


In [8]:
import pickle
from sentence_transformers import SentenceTransformer
import torch
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

c:\Users\georg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cache_dir = "C:\\LLMs"
sentence_model = SentenceTransformer('all-mpnet-base-v2', cache_folder = cache_dir)
# sentence_model = SentenceTransformer('nvidia/NV-Embed-v2', trust_remote_code=True, cache_folder = cache_dir)

c:\Users\georg\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
with open('query_to_time_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

print(dataset)

[{'question': "Are you interested in the following article? Website: msn.com\nTitle: Race Rewind: Phoenix heats up and Denny delivers\nDescription: Relive Denny Hamlin's incredible day at ISM Raceway in this week's Monster Energy NASCAR Cup Series Race Rewind.", 'response_time': 7.539, 'prolific_id': '5d23460ad4c57900192b129c'}, {'question': "Are you interested in the following article? Website: msn.com\nTitle: 15 Things Dog Shelters Need You to Know\nDescription: When you're looking for your new best friend, staffers and volunteers at dog shelters can help.", 'response_time': 12.535, 'prolific_id': '5d23460ad4c57900192b129c'}, {'question': "Are you interested in the following article? Website: msn.com\nTitle: 2021 VW Golf GTI Caught At The 'Ring Perfecting Hot Hatch Recipe\nDescription: One of the very few eighth-gen Golfs coming to the U.S.", 'response_time': 5.969, 'prolific_id': '5d23460ad4c57900192b129c'}, {'question': 'Are you interested in the following article? Website: msn.com

In [15]:
for i, data in tqdm(enumerate(dataset)):
    dataset[i]['embedding'] = sentence_model.encode(data['question'])

3013it [02:41, 18.71it/s]


In [16]:
with open('query_to_time_embedding_dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)